In [1]:
import pandas as pd

In [2]:
data_df = pd.read_json('logins.json')

In [3]:
data_df.info

<bound method DataFrame.info of                login_time
0     1970-01-01 20:13:18
1     1970-01-01 20:16:10
2     1970-01-01 20:16:37
3     1970-01-01 20:16:36
4     1970-01-01 20:26:21
...                   ...
93137 1970-04-13 18:50:19
93138 1970-04-13 18:43:56
93139 1970-04-13 18:54:02
93140 1970-04-13 18:57:38
93141 1970-04-13 18:54:23

[93142 rows x 1 columns]>

In [4]:
data_df['login_time'].dtype

dtype('<M8[ns]')

In [5]:
data_df.set_index('login_time', inplace = True)

In [6]:
data_df.head()

""
login_time
1970-01-01 20:13:18
1970-01-01 20:16:10
1970-01-01 20:16:37
1970-01-01 20:16:36
1970-01-01 20:26:21


In [7]:
data_df.index.dtype

dtype('<M8[ns]')

In [11]:
resampled_df = data_df.resample('15T').count()
resampled_df.head()

""
login_time
1970-01-01 20:00:00
1970-01-01 20:15:00
1970-01-01 20:30:00
1970-01-01 20:45:00
1970-01-01 21:00:00
